# AgbCloudToolkit

This will help you get started with the AGB Cloud [toolkit](/docs/concepts/tools/#toolkits). For detailed documentation of all AgbIntegrationToolkit features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/agent_toolkits/langchain_agb_cloud.agent_toolkits.AgbCloud.toolkit.AgbCloudToolkit.html).

The AGB platform integration provides tools for file operations, code execution, and command execution within a secure cloud environment.

To enable automated tracing of individual tools, set your [LangSmith](https://docs.smith.langchain.com/) API key:

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

## Setup

To use the AGB integration, you need to get an API key from [AGB Cloud](https://agb.cloud/) and configure it as an environment variable:

```bash
export AGB_API_KEY="your-api-key"
```

### Prerequisites

- **AGB Cloud account**: Register your free AGB Cloud account [here](https://agb.cloud/).
- **AGB API token**: Learn how to get your API token from the AGB platform dashboard.
- **OpenAI API key**: For using OpenAI models with the agent.

In [ ]:
import os

os.environ["AGB_API_KEY"] = "your-agb-api-key"
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

### Installation

This toolkit lives in the `langchain-agb-integration` package. You also need to install the `agb` package to interact with the AGB platform:

In [ ]:
%pip install -qU langchain-agb-integration agb

In [ ]:
%pip install -qU langchain-openai langgraph

## Instantiation

Now we can instantiate our toolkit:

In [ ]:
from agb import AGB
from langchain_agb_integration import AgbIntegrationToolkit

# Create AGB session
agb = AGB()
result = agb.create()
session = result.session

toolkit = AgbIntegrationToolkit(session=session)

## Tools

View available tools:

In [ ]:
tools = toolkit.get_tools()

The toolkit provides four main tools:

1. **WriteFileTool**: Write content to a file in the AGB session with either overwrite or append mode.
2. **ReadFileTool**: Read content from a file in the AGB session.
3. **RunCodeTool**: Execute Python or JavaScript code in a secure cloud environment with configurable timeout.
4. **ExecuteCommandTool**: Execute shell commands in the AGB session with timeout control.

## Use within an agent

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can execute code and commands in a secure cloud environment."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
example_query = "Create a Python script that prints 'Hello from AGB Cloud!' and execute it."

result = agent_executor.invoke({"input": example_query})
print(result["output"])

## Individual Tool Usage

You can also use tools individually:

In [ ]:
from langchain_agb_integration.tools import WriteFileTool, ReadFileTool, RunCodeTool, ExecuteCommandTool

# Create individual tools
write_tool = WriteFileTool(session=session)
read_tool = ReadFileTool(session=session)
code_tool = RunCodeTool(session=session)
command_tool = ExecuteCommandTool(session=session)

# Use tools
write_tool.invoke({"path": "/tmp/test.txt", "content": "Hello World", "mode": "overwrite"})
read_tool.invoke({"path": "/tmp/test.txt"})
code_tool.invoke({"code": "print('Hello from Python!')", "language": "python"})
command_tool.invoke({"command": "ls -la", "timeout_ms": 1000})